In [1]:
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
text_document=loader.load()
text_document

[Document(metadata={'source': 'speech.txt'}, page_content='The most famous speech in history is widely considered to be "I Have a Dream" by Martin Luther King Jr., delivered on August 28, 1963, during the March on Washington for Jobs and Freedom.  This powerful address, calling for an end to racism and equality for all Americans, remains a defining moment in the Civil Rights Movement and continues to inspire global efforts for justice and human rights. It is consistently ranked at the top of lists such as Top 100 American Speeches of the 20th Century by the University of Wisconsin-Madison and Texas A&M University, and is recognized as the most influential and enduring speech in American history.')]

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [3]:
from  langchain_community.document_loaders import WebBaseLoader
import bs4

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain_community.document_loaders import WebBaseLoader
from bs4 import SoupStrainer

loader = WebBaseLoader(
    web_paths=[
        "https://lilianweng.github.io/posts/2024-07-07-hallucination/"
    ],
    bs_kwargs=dict(
        parse_only=SoupStrainer(
            class_=("post-title", "post-content", "post-header")
        )
    )
)

docs = loader.load()
print(docs[0].page_content[:500])




      Extrinsic Hallucinations in LLMs
    
Date: July 7, 2024  |  Estimated Reading Time: 29 min  |  Author: Lilian Weng


Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=50
)

documents = text_splitter.split_documents(docs)

documents[:5]


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/'}, page_content='Extrinsic Hallucinations in LLMs\n    \nDate: July 7, 2024  |  Estimated Reading Time: 29 min  |  Author: Lilian Weng\n\n\nHallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation.

In [6]:
from langchain_ollama import OllamaEmbeddingsfrom langchain_community.vectorstores import FAISSembeddings = OllamaEmbeddings(    model="nomic-embed-text")db = FAISS.from_documents(    documents[:20],    embedding=embeddings)retriever=db.as_retriever()retriever

In [7]:
query="What is hallucination"
result=db.similarity_search(query)
result[1].page_content

'This post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.\nWhat Causes Hallucinations?#\nGiven a standard deployable LLM goes through pre-training and fine-tuning for alignment and other improvements, let us consider causes at both stages.\nPre-training Data Issues#\nThe volume of the pre-training data corpus is enormous, as it is supposed to represent world knowledge in all available written forms. Data crawled from the public Internet is the most common choice and thus out-of-date, missing, or incorrect information is expected. As the model may incorrectly memorize this information by simply maximizing the log-likelihood, we would expect the model to make mistakes.\nFine-tuning New Knowledge#\nFine-tuning a pre-trained LLM via supervised fine-tuning and RLHF is a common technique for improving certain capabilities of the model like instruction following. Introducing new knowledge at t

In [9]:
db

In [11]:
from langchain_community.llms import Ollama
llm=Ollama(model="llama2")
llm

Ollama()

In [15]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template('''
Answer the following question based only on the providing detailed answer.
i will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>
Question:{input}''')

In [19]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

ModuleNotFoundError: No module named 'langchain.chains'

In [ ]:
retriever=db.as_retriever()
retriever
